In [ ]:
import json
import math
import pytz
from datetime import datetime,timedelta,timezone
from dateutil.tz import tzutc
from dateutil.relativedelta import relativedelta
import os
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import math
import csv
from scipy.interpolate import interp1d
from scipy.stats import pearsonr, mode,skew,kurtosis
from scipy.signal import savgol_filter
import xgboost as xgb
import sklearn
from sklearn.metrics import mean_squared_error,plot_confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit
from imblearn.over_sampling import SMOTE
from scipy.interpolate import UnivariateSpline
from sklearn.utils.multiclass import type_of_target

addDataPrefix='/Users/sorush/My Drive/Documents/Educational/TAMU/Research/Trial/Data/11-5-21-11-15-21'
if not os.path.exists(addDataPrefix):
    addDataPrefix='/home/grads/s/sorush.omidvar/CGMDataset/Trial/Data/11-5-21-11-15-21'
if not os.path.exists(addDataPrefix):
   addDataPrefix = 'C:\\GDrive\\Documents\\Educational\\TAMU\\Research\\Trial\\Data\\11-5-21-11-15-21'

addUserInput=os.path.join(addDataPrefix,'User inputted')
addHKCM=os.path.join(addDataPrefix,'hk+cm')
addCGM=os.path.join(addDataPrefix,'CGM')
addE4=os.path.join(addDataPrefix,'E4')

exempts=['p2']

pd.options.mode.chained_assignment = None  # default='warn'
plt.style.use({'figure.facecolor':'white'})

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 50)

In [ ]:

START_OF_TRIAL = datetime.strptime('11 06 2021-02:00:00', '%m %d %Y-%H:%M:%S')#to handle the daylight saving issue in apple watches
END_OF_TRIAL = datetime.strptime('11 15 2021-00:00:00', '%m %d %Y-%H:%M:%S')  

if os.path.exists(os.path.join(addDataPrefix,'All_meals.pkl')):
    os.remove(os.path.join(addDataPrefix,'All_meals.pkl'))
os.chdir(addUserInput)
if not os.path.exists(os.path.join(addDataPrefix,'All_meals.pkl')):    
    df=[]
    for root, dirs, files in os.walk(addUserInput):
        for file in files:
            if '.csv' in file.lower():
                if 'meals' in file.lower():
                    participantName=file[:file.find('Meals')]
                    if participantName in exempts:
                        print("Exemption...",file)
                        continue
                    print("Reading ...",file)
                    dfTemp = pd.read_csv(file)
                    dfTemp.insert(0,'Participant',participantName)
                    dfTemp.rename(columns={'startTime':'StartTime'}, inplace=True)
                    dfTemp['StartTime']=pd.to_datetime(dfTemp['StartTime'])
                    dfTemp['FinishTime']=pd.to_datetime(dfTemp['FinishTime'])

                    dfTemp['StartTime']-=pd.DateOffset(hours=5)#fixing the time zone issue
                    dfTemp['FinishTime']-=pd.DateOffset(hours=5)#fixing the time zone issue            
                    dfTemp.sort_values(["Participant",'StartTime'],ascending = (True, True),inplace=True)
                    if len(dfTemp.columns)!=10:
                        print("MAYDAY. Error in reading csv")
                        break
                    if len(df)!=0:
                        frames=[dfTemp,df]
                        df=pd.concat(frames)
                    else:
                        df=dfTemp
    print("reading is done")
    dfMeal=df    
    dfMeal=dfMeal[dfMeal['StartTime']>=START_OF_TRIAL]
    dfMeal=dfMeal[dfMeal['FinishTime']<END_OF_TRIAL]
    print("Meal database is limited to the trial period")
    dfMeal.to_pickle(os.path.join(addDataPrefix,'All_meals.pkl')) 
else:
    dfMeal=pd.read_pickle(os.path.join(addDataPrefix,'All_meals.pkl')) 

def pdInterpolation(dfTemp):
    index=dfTemp['Time']
    seriesParticipant = pd.Series(dfTemp['Abbot'].to_list(), index=index)
    seriesParticipant=seriesParticipant.resample('1T').asfreq()
    seriesParticipant.interpolate(method='polynomial',order=3,inplace=True)
    tempTime=seriesParticipant.index
    tempVal=seriesParticipant.values
    dfTemp=pd.DataFrame(zip(tempTime,tempVal),columns=['Time','Abbot'])
    return dfTemp

if os.path.exists(os.path.join(addDataPrefix,'All_cgm.pkl')):
    os.remove(os.path.join(addDataPrefix,'All_cgm.pkl'))
if not os.path.exists(os.path.join(addDataPrefix,'All_cgm.pkl')):
    os.chdir(addCGM)
    dfCGM=[]
    for root, dirs, files in os.walk(addCGM):
        for file in files:
            if '.txt' in file.lower():
                if '_fl' in file.lower():
                    participantName=file[:file.find('_fl')]
                    if participantName in exempts:
                        print("Exemption...",file)
                        continue
                    print("Reading ...",file)
                    dfTemp = pd.read_csv(file,sep="\t",skiprows=1)
                    if len(dfTemp.columns)!=4:
                        print("MAYDAY. Error in reading csv")
                        break               
                    dfTemp.columns.values[0] = "ID"
                    dfTemp.columns.values[1] = "Time"
                    dfTemp.columns.values[2] = "Record"
                    dfTemp.columns.values[3] = "Abbot"
                    dfTemp.drop(columns=["ID",'Record'],inplace=True)                
                    dfTemp['Time']=pd.to_datetime(dfTemp['Time'])
                    dfTemp['Abbot']=pd.to_numeric(dfTemp['Abbot'])
                    dfTemp.sort_values(["Time"],ascending = (True),inplace=True)
                    dfTemp=pdInterpolation(dfTemp)
                    dfTemp.insert(0,'Participant',participantName)
                    if len(dfTemp.columns)!=3:
                        print("MAYDAY. Error in processing csv")
                        break                    
                    if len(dfCGM)!=0:
                        frames=[dfTemp,dfCGM]
                        dfCGM=pd.concat(frames)
                    else:
                        dfCGM=dfTemp
    print("reading is done")
    dfCGM=dfCGM[dfCGM['Time']>=START_OF_TRIAL]
    dfCGM=dfCGM[dfCGM['Time']<END_OF_TRIAL]
    print("CGM database is limited to the trial period")
    dfCGM.to_pickle(os.path.join(addDataPrefix,'All_cgm.pkl')) 
else:
    dfCGM=pd.read_pickle(os.path.join(addDataPrefix,'All_cgm.pkl'))


  
# if os.path.exists(os.path.join(addDataPrefix,'All_cm.pkl')):
#     os.remove(os.path.join(addDataPrefix,'All_cm.pkl'))
os.chdir(addHKCM)
if not os.path.exists(os.path.join(addDataPrefix,'All_cm.pkl')):
    df=[]
    for root, dirs, files in os.walk(addHKCM):
        for file in files:
            if '.csv' in file.lower():
                if 'cm' in file.lower() and 'modified' in file.lower():
                    participantName=file[:file.find('_cm')]
                    if participantName in exempts:
                        print("Exemption...",file)
                        continue
                    print("Reading ...",file)
                    dfTemp=pd.read_csv(file)
                    print("File is read")
                    dfTemp['Time']=pd.to_datetime(dfTemp['Time'])
                    dfTemp.insert(0,'Participant',participantName)
                    dfTemp.insert(len(dfTemp.columns),'|Ax|+|Ay|+|Az|',dfTemp['Ax'].abs()+dfTemp['Ay'].abs()+dfTemp['Az'].abs()+0.001)#this is to avoid 0 later on for feature calculation
                    dfTemp.insert(len(dfTemp.columns),'|Yaw|+|Roll|+|Pitch|',dfTemp['Yaw'].abs()+dfTemp['Roll'].abs()+dfTemp['Pitch'].abs())
                    dfTemp.insert(len(dfTemp.columns),'RotationalToLinear',dfTemp['|Yaw|+|Roll|+|Pitch|']/dfTemp['|Ax|+|Ay|+|Az|'])
                    print("modified")
                    dfTemp.sort_values(['Time'],ascending = (True),inplace=True)
                    print("sorted")
                    if len(dfTemp.columns)!=14:
                        print("MAYDAY. Error in reading csv")
                        print(dfTemp.columns)
                        break
                    if len(df)!=0:
                        frames=[dfTemp,df]
                        df=pd.concat(frames)
                    else:
                        df=dfTemp
    dfCM=df
    print("Processing is done")
    dfCM=dfCM[dfCM['Time']>=START_OF_TRIAL]
    dfCM=dfCM[dfCM['Time']<END_OF_TRIAL]
    print("CM database is limited to the trial period")
    dfCM.to_pickle(os.path.join(addDataPrefix,'All_cm.pkl')) 
else:
    dfCM = pd.read_pickle(os.path.join(addDataPrefix,'All_cm.pkl'))


# if os.path.exists(os.path.join(addDataPrefix,'All_E4.pkl')):
#     os.remove(os.path.join(addDataPrefix,'All_E4.pkl'))
os.chdir(addE4)
# fields=['ACC','BVP','EDA','HR','IBI','TEMP']
fields=['BVP','EDA','HR','TEMP']
if not os.path.exists(os.path.join(addDataPrefix,'All_E4.pkl')):    
    dfE4=[]
    for root, dirs, files in os.walk(addE4):
        for file in files:
            if '.csv' in file.lower():
                participantName=root[root.find('E4')+3:]
                participantName=participantName[:2]
                field=file[:file.find('.csv')]
                if field not in fields:
                    print("File name does not comply with analyzed fields",file)
                    continue
                print(participantName,field)
                if participantName in exempts:
                    print("Exemption...",file)
                    continue
                print("Reading ...",file)
                os.chdir(root)
                dfTemp = pd.read_csv(file,header=None)
                # if field=='ACC':
                #     assert len(dfTemp.columns)==3
                #     timeBase=dfTemp.iloc[0,0]
                #     timeStep=1/dfTemp.iloc[1,0]
                #     dfTemp.drop([0,1],inplace=True)
                #     dfTemp.rename(columns={0:'Data1',1:'Data2',2:'Data3'}, inplace=True)#x,y,z for data1,data2,data3
                #     timeTemp=[]
                #     for counter in range(len(dfTemp)):
                #         timeTemp.append(timeBase+counter*timeStep)
                #     dfTemp.insert(0,'Time',timeTemp)
                #     dfTemp.insert(0,'Field',"Acceleration")
                #     dfTemp['Time'] = pd.to_datetime(dfTemp['Time'],unit='s')
                if field=='BVP':
                    assert len(dfTemp.columns)==1
                    timeBase=dfTemp.iloc[0,0]
                    timeStep=1/dfTemp.iloc[1,0]
                    dfTemp.drop([0,1],inplace=True)
                    dfTemp.rename(columns={0:'Data1'}, inplace=True)
                    dfTemp["Data2"]=""
                    dfTemp["Data3"]=""
                    timeTemp=[]
                    for counter in range(len(dfTemp)):
                        timeTemp.append(timeBase+counter*timeStep)
                    dfTemp.insert(0,'Time',timeTemp)
                    dfTemp.insert(0,'Field',"BVP")
                    dfTemp['Time'] = pd.to_datetime(dfTemp['Time'],unit='s')
                elif field=='HR':
                    assert len(dfTemp.columns)==1
                    timeBase=dfTemp.iloc[0,0]
                    timeStep=1/dfTemp.iloc[1,0]
                    dfTemp.drop([0,1],inplace=True)
                    dfTemp.rename(columns={0:'Data1'}, inplace=True)
                    dfTemp["Data2"]=""
                    dfTemp["Data3"]=""
                    timeTemp=[]
                    for counter in range(len(dfTemp)):
                        timeTemp.append(timeBase+counter*timeStep)
                    dfTemp.insert(0,'Time',timeTemp)
                    dfTemp.insert(0,'Field',"HR")
                    dfTemp['Time'] = pd.to_datetime(dfTemp['Time'],unit='s')                
                elif field=='EDA':
                    assert len(dfTemp.columns)==1
                    timeBase=dfTemp.iloc[0,0]
                    timeStep=1/dfTemp.iloc[1,0]
                    dfTemp.drop([0,1],inplace=True)
                    dfTemp.rename(columns={0:'Data1'}, inplace=True)
                    dfTemp["Data2"]=""
                    dfTemp["Data3"]=""                    
                    timeTemp=[]
                    for counter in range(len(dfTemp)):
                        timeTemp.append(timeBase+counter*timeStep)
                    dfTemp.insert(0,'Time',timeTemp)
                    dfTemp.insert(0,'Field',"EDA")
                    dfTemp['Time'] = pd.to_datetime(dfTemp['Time'],unit='s')
                # elif field=='IBI':
                #     assert len(dfTemp.columns)==2
                #     timeBase=dfTemp.iloc[0,0]
                #     dfTemp.drop([0],inplace=True)
                #     dfTemp.rename(columns={0:'Time',1:'Data1'}, inplace=True)
                #     dfTemp["Data2"]=""
                #     dfTemp["Data3"]=""
                #     timeTemp=[]
                #     dfTemp['Time']+=timeBase
                #     dfTemp.insert(0,'Field',"IBI")
                #     dfTemp['Time'] = pd.to_datetime(dfTemp['Time'],unit='s')
                elif field=='TEMP':
                    assert len(dfTemp.columns)==1
                    timeBase=dfTemp.iloc[0,0]
                    timeStep=1/dfTemp.iloc[1,0]
                    dfTemp.drop([0,1],inplace=True)
                    dfTemp.rename(columns={0:'Data1'}, inplace=True)
                    dfTemp["Data2"]=""
                    dfTemp["Data3"]=""                    
                    timeTemp=[]
                    for counter in range(len(dfTemp)):
                        timeTemp.append(timeBase+counter*timeStep)
                    dfTemp.insert(0,'Time',timeTemp)
                    dfTemp.insert(0,'Field',"Temperature")
                    dfTemp['Time'] = pd.to_datetime(dfTemp['Time'],unit='s')             
                dfTemp.insert(0,'Participant',participantName)

                dfTemp['Time']-=pd.DateOffset(hours=6)#Empatica records in GMT and also during the trial we had daylight saving
                dfTemp.sort_values(["Participant",'Field',"Time"],ascending = (True,True, True),inplace=True)
                if len(dfTemp.columns)!=6:
                    print("MAYDAY. Error in reading csv")
                    break
                if len(dfE4)!=0:
                    frames=[dfTemp,dfE4]
                    dfE4=pd.concat(frames)
                else:
                    dfE4=dfTemp
                
    print("reading is done")  
    dfE4=dfE4[dfE4['Time']>=START_OF_TRIAL]
    dfE4=dfE4[dfE4['Time']<END_OF_TRIAL]
    print("E4 database is limited to the trial period")
    dfE4.to_pickle(os.path.join(addDataPrefix,'All_E4.pkl')) 
else:
    dfE4=pd.read_pickle(os.path.join(addDataPrefix,'All_E4.pkl'))


In [ ]:
MINIMUM_POINT_CM=100
OUTTER_WINDOW_LENGTH=timedelta(minutes=180)
OUTTER_WINDOW_STEP=timedelta(minutes=90)
EATING_PORTION=timedelta(minutes=90)
INNER_WINDOW_LENGTH=timedelta(minutes=1)


def e4Reporter(df):
    topics=['BVP','EDA','HR','Temperature']
    report=[]
    for topic in topics:
        dfTemp=df[df['Field']==topic]
        if(len(dfTemp)<MINIMUM_POINT_CM):
            report.append('Nan')
        else:
            val=dfTemp['Data1'].mean()
            report.append(val)
    return report

def motionCalculator(df):
    f1=df['RotationalToLinear'].mean()
    f2=df['|Ax|+|Ay|+|Az|'].mean()
    f5=df['|Yaw|+|Roll|+|Pitch|'].mean()
    return [f1,f2,f5]

def featureExtractor(dataList):
    dataList=np.asarray(dataList).astype(float)
    result=[]
    dataDim=dataList.ndim
    if dataDim>1:
        for counter in range(dataList.shape[1]):
            meanVal=np.nanmean(dataList[:,counter],axis=0)
            stdVal=np.nanstd(dataList[:,counter],axis=0)
            minVal=np.nanmin(dataList[:,counter],axis=0)
            maxVal=np.nanmax(dataList[:,counter],axis=0)
            rangeVal=maxVal-minVal
            # skewnessVal=skew(dataList[:,counter],nan_policy='omit',axis=0)
            # kurtosisVal=kurtosis(dataList[:,counter],nan_policy='omit',axis=0)
            result.extend([rangeVal, meanVal, stdVal, minVal, maxVal])
    else:
        meanVal=np.nanmean(dataList)
        stdVal=np.nanstd(dataList)
        minVal=np.nanmin(dataList)
        maxVal=np.nanmax(dataList)
        rangeVal=maxVal-minVal
        # skewnessVal=skew(dataList,nan_policy='omit')
        # kurtosisVal=kurtosis(dataList,nan_policy='omit')
        result.extend([rangeVal, meanVal, stdVal, minVal, maxVal])        
    return result


participants=dfCM['Participant'].to_list()
participants=list(set(participants))
participantDataList=[]
skippedWindows=0

if os.path.exists(os.path.join(addDataPrefix,'Features.pkl')):
    os.remove(os.path.join(addDataPrefix,'Features.pkl'))
if not os.path.exists(os.path.join(addDataPrefix,'Features.pkl')):
    for participant in participants:
        outterWindowStart=START_OF_TRIAL
        outterWindowEnd=outterWindowStart+OUTTER_WINDOW_LENGTH        
        print("Participant ",participant," is started")
        dfParticipantCM=dfCM[dfCM['Participant']==participant]
        dfParticipantMeal=dfMeal[dfMeal['Participant']==participant]
        dfParticipantCGM=dfCGM[dfCGM['Participant']==participant]
        dfParticipantE4=dfE4[dfE4['Participant']==participant]
        print("CM size for participant",len(dfParticipantCM))
        innerWindowStart=outterWindowStart
        innerWindowEnd=innerWindowStart+INNER_WINDOW_LENGTH        
        for i in tqdm(range(0,int(((END_OF_TRIAL-START_OF_TRIAL).total_seconds())/(OUTTER_WINDOW_LENGTH.total_seconds())),1)):
            dfTempMeal=dfParticipantMeal[(dfParticipantMeal['StartTime']>=outterWindowStart) & (dfParticipantMeal['StartTime']<=outterWindowStart+EATING_PORTION)]
            tempList=[]
            tempListCM=[]
            tempListE4=[]
            tempListCGM=[]
            tempListInfo=[outterWindowStart,outterWindowEnd,participant]
            skippedFlag=True
            for j in range(0,int((OUTTER_WINDOW_LENGTH.total_seconds())/(INNER_WINDOW_LENGTH.total_seconds())),1):
                dfTempCM=dfParticipantCM[(dfParticipantCM['Time']>=innerWindowStart) & (dfParticipantCM['Time']<innerWindowEnd)]
                if(len(dfTempCM)<MINIMUM_POINT_CM):
                    tempListCM.append(['Nan','Nan','Nan'])
                else:
                    tempListCM.append(motionCalculator(dfTempCM))

                dfTempE4=dfParticipantE4[(dfParticipantE4['Time']>=innerWindowStart) & (dfParticipantE4['Time']<innerWindowEnd)]
                if(len(dfTempE4)<MINIMUM_POINT_CM):
                    tempListE4.append(['Nan','Nan','Nan','Nan'])
                else:
                    tempListE4.append(e4Reporter(dfTempE4))
                
                if(len(dfTempCM)>MINIMUM_POINT_CM and len(dfTempE4)>MINIMUM_POINT_CM):
                    skippedFlag=False
                innerWindowStart+=INNER_WINDOW_LENGTH
                innerWindowEnd+=INNER_WINDOW_LENGTH
            if len(dfTempMeal)>0:
                mealFlag=1
            else:
                mealFlag=0
                
            carbs=dfTempMeal['Carbs'].sum()
            protein=dfTempMeal['Protein'].sum()
            fat=dfTempMeal['Fat'].sum()

            if skippedFlag:
                skippedWindows+=1
            else:
                assert len(tempListCM)==int((OUTTER_WINDOW_LENGTH.total_seconds())/(INNER_WINDOW_LENGTH.total_seconds()))
                tempListCM=featureExtractor(tempListCM)
                tempList.extend(tempListCM) #5*3=15

                assert len(tempListE4)==int((OUTTER_WINDOW_LENGTH.total_seconds())/(INNER_WINDOW_LENGTH.total_seconds()))
                tempListE4=featureExtractor(tempListE4)
                tempList.extend(tempListE4) #5*4=20

                dfTempCGM=dfParticipantCGM[(dfParticipantCGM['Time']>=outterWindowStart) & (dfParticipantCGM['Time']<outterWindowEnd)]
                tempListCGM=dfTempCGM['Abbot'].to_list()
                tempListCGM=featureExtractor(tempListCGM)
                tempList.extend(tempListCGM) #5

                tempList.extend(tempListInfo) #3
                tempList.append(carbs)
                tempList.append(fat)
                tempList.append(protein)
                tempList.append(mealFlag) #1
                assert len(tempList)==5*3+5*4+5+3+4
                participantDataList.append(tempList)
            outterWindowStart+=OUTTER_WINDOW_STEP
            outterWindowEnd+=OUTTER_WINDOW_STEP
    participantDataArray=np.asarray(participantDataList)
    columnTopics=['F1','F2','F5','BVP','EDA','HR','Temperature','CGM']
    columnStats=['Range','Mean','Std','Min','Max']
    columns=[]
    for topic in columnTopics:
        for stat in columnStats:
            columns.append(topic+stat)
    columns.extend(['Start','End','Participant','Carb','Fat','Protein','MealLabel'])
    dfFeatures=pd.DataFrame(participantDataArray,columns=columns)
    dfFeatures.to_pickle(os.path.join(addDataPrefix,'Features.pkl')) 
else:
    dfFeatures = pd.read_pickle(os.path.join(addDataPrefix,'Features.pkl'))


In [ ]:
print(dfFeatures)

In [ ]:
def STMI_XGBoost(xTrain,xVal,xTest,yTrain,yVal,yTest):
    f1ScoreBest=-1
    for maxDepth in tqdm(np.arange(2,8,1)):
        for estimator in np.arange(50,200,50):
            for posWeight in np.arange(1,50,5):
                clf = xgb.XGBClassifier(scale_pos_weight = posWeight, n_jobs=18,n_estimators=estimator,max_depth=maxDepth, objective = "binary:logistic", eval_metric = "error")
                clf.fit(xTrain,yTrain)
                predictionsVal = clf.predict(xVal)
                # predictionsVal = clf.predict_proba(xVal)
                # predictionsVal=predictionsVal[:,1]
                # predictionsVal[predictionsVal>=threshold]=1
                # predictionsVal[predictionsVal<threshold]=0
                
                confMatrix=sklearn.metrics.confusion_matrix(yVal,predictionsVal)
                accuracy=sklearn.metrics.accuracy_score(yVal,predictionsVal)
                recall=sklearn.metrics.recall_score(yVal,predictionsVal)
                precision=sklearn.metrics.precision_score(yVal,predictionsVal)
                f1Score=sklearn.metrics.f1_score(yVal,predictionsVal)

                if f1Score>f1ScoreBest:
                    confMatrixBest=confMatrix
                    accuracyBest=accuracy
                    modelBest=clf
                    recallBest=recall
                    precisionBest=precision
                    f1ScoreBest=f1Score
    # print("***********Val:")
    # print(confMatrixBest)
    # print("Accuracy:",np.round(100*accuracyBest,0),"Recall:",np.round(100*recallBest,0),"Precision:",np.round(100*precisionBest,0))
    # print("***********Test:")
    predictionsTest=modelBest.predict(xTest)
    # predictionsTest=modelBest.predict_proba(xTest)
    # predictionsTest=predictionsTest[:,1]
    # predictionsTest[predictionsTest>=thresholdBest]=1
    # predictionsTest[predictionsTest<thresholdBest]=0    
    
    confMatrix=sklearn.metrics.confusion_matrix(yTest,predictionsTest)
    accuracy=sklearn.metrics.accuracy_score(yTest,predictionsTest)
    recall=sklearn.metrics.recall_score(yTest,predictionsTest)
    precision=sklearn.metrics.precision_score(yTest,predictionsTest)
    
    print(confMatrix)
    print("Accuracy:",np.round(100*accuracy,0),"Recall:",np.round(100*recall,0),"Precision:",np.round(100*precision,0))
    plot_confusion_matrix(modelBest, xTest, yTest,cmap='bone') 

def dataBalancer(xTrain,xVal,yTrain,yVal):
    oversample = SMOTE()
    xVal, yVal = oversample.fit_resample(xVal, yVal)
    xTrain, yTrain = oversample.fit_resample(xTrain, yTrain)

    return xTrain,xVal,yTrain,yVal

def testTrainSplitFunc(data,randomSeed,normalFlag):
    participants=data[:,data.shape[1]-5]
    participants=list(set(participants))
    print(participants)
    for participant in participants:
        if participant !=3:#####################################################################
            continue
        
        print("Participant:",participant)
        indxList=[]
        for counter in range(data.shape[0]):
            if(data[counter,data.shape[1]-5]==participant):
                indxList.append(counter)
        dataParticipant=data[indxList,:]
                     
        carb=dataParticipant[:,dataParticipant.shape[1]-4]
        fat=dataParticipant[:,dataParticipant.shape[1]-3]
        protein=dataParticipant[:,dataParticipant.shape[1]-2]     
            
        windowStarts=dataParticipant[:,dataParticipant.shape[1]-7]
        windowEnds=dataParticipant[:,dataParticipant.shape[1]-6]
        
        dataX=dataParticipant[:,0:dataParticipant.shape[1]-7]
        dataY=dataParticipant[:,dataParticipant.shape[1]-1]
        dataY=dataY.astype(float)
        dataX=dataX.astype(float)
        if(normalFlag):
            dataX=dataX-dataX.mean(axis=0)
            dataX/=dataX.std(axis=0)
        
        stratidiedSampling = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=randomSeed)
        for trainIndex, testIndex in stratidiedSampling.split(dataX, dataY):
            xTrain,xTest=dataX[trainIndex],dataX[testIndex]
            yTrain,yTest=dataY[trainIndex],dataY[testIndex]

        stratidiedSampling = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=randomSeed)
        for trainIndex, valIndex in stratidiedSampling.split(xTrain, yTrain):
            xTrain,xVal=dataX[trainIndex],dataX[valIndex]
            yTrain,yVal=dataY[trainIndex],dataY[valIndex]
        print(np.sum(yTrain),np.sum(yVal),np.sum(yTest))
        # xTrain,xVal,yTrain,yVal=dataBalancer(xTrain,xVal,yTrain,yVal)
        STMI_XGBoost(xTrain,xVal,xTest,yTrain,yVal,yTest)




combinations=[['CGM'],['CGM','F1','F2','F5'],['CGM','BVP','EDA','HR','Temperature'],['CGM','BVP','EDA','HR','Temperature','F1','F2','F3']]
columns=dfFeatures.columns
for combination in combinations:
    columnList=['Start','End','Participant','Carb','Fat','Protein','MealLabel']
    for topic in combination:
        for column in columns:
            if topic in column:
                columnList.append(column)
                
    dfCombination = dfFeatures[dfFeatures.columns.intersection(columnList)]
    participantDataArray=dfCombination.to_numpy()
    randomSeed=random.randrange(50)
    print("----------------------")
    print("Combination:",combination)
    NORMALIZED_FLAG=False
    testTrainSplitFunc(participantDataArray,randomSeed,NORMALIZED_FLAG)

# print("**********************************")
# print("**********************************")
# print("With NORMALIZATION")
# NORMALIZED_FLAG=True
# testTrainSplitFunc(participantDataArray,randomSeed,NORMALIZED_FLAG)

In [ ]:
print(participantDataArray)